In [ ]:
import os
path = '/Users/edwardtyler/Desktop/eye-images_2'

Normal = os.path.join(path,'1_normal')
Cataract = os.path.join(path,'2_cataract')
Glaucoma = os.path.join(path, '2_glaucoma')
print(len(os.listdir(Normal)))
print(len(os.listdir(Cataract)))
print(len(os.listdir(Glaucoma)))

import numpy as np
import cv2
from sklearn.utils import class_weight

def load_images(directories, n_images=900000):
    """
    Reads in images and assigns class labels
    Parameters:
        directories: A list of the sub-directories
        n_images:    The maximum number of images to load from each directory
    Returns:
        images (numpy.ndarray) : Image data
        label (numpy.ndarray      : Labels of each image
    """
    images = []
    labels = []
    for label, sub_dir in enumerate(directories):
        num=1
        for file_name in os.listdir(sub_dir):
            if num > n_images:
                break
            img_path = os.path.join(sub_dir, file_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (100, 100))  # Resize to a smaller, consistent shape
                #gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #images.append(gray_image)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
                labels.append(label)  
                num+=1
    return np.array(images), np.array(labels)

images, labels = load_images([Normal, Cataract, Glaucoma], 500)

In [ ]:
def create_balanced_dataset(images, labels):
    """
    Balance dataset by computing class weights for each class
    """
    labels = labels.flatten()
    class_weights = class_weight.compute_class_weight(
        class_weight='balanced', classes=np.unique(labels), y=labels
    )
    print(f"Class weights: {class_weights}")
    class_weights_dict = dict(zip(np.unique(labels), class_weights))
    
    return images, labels, class_weights_dict


balanced_images, balanced_labels, class_weights_dict = create_balanced_dataset(images, labels)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Rescaling

model = Sequential( [
 Rescaling(1./255, input_shape=(100, 100, 3)),   # Rescale the colour to be between 0 and 1 
 Conv2D(30, (3, 3), activation='relu'), # Set number of filters to 30 
 MaxPooling2D(pool_size=(2, 2)),
 Conv2D(60, (3, 3), activation='relu'), # Increase number of filters
 MaxPooling2D(pool_size=(2, 2)),
 Conv2D(120, (3, 3), activation='relu'), # Increase number of filters
 MaxPooling2D(pool_size=(2, 2)),
 Conv2D(240, (3, 3), activation='relu'), # Increase number of filters
 MaxPooling2D(pool_size=(2, 2)),
 Flatten(),
 Dense(120, activation='relu'),  # Put into a dense layer size
 Dense(3, activation='softmax')  

] )

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=142)

import tensorflow
y_train_cat = tensorflow.keras.utils.to_categorical(y_train, num_classes=3)
# also for the test set
y_test_cat = tensorflow.keras.utils.to_categorical(y_test, num_classes=3)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values
    rotation_range=10,         # Rotate images by up to 10 degrees
    width_shift_range=0.1,     # Shift image width by 10%
    height_shift_range=0.1,    # Shift image height by 10%
    shear_range=0.1,           # Shear transformation
    zoom_range=0.1,            # Zoom in/out by 10%
    horizontal_flip=True,      # Flip images horizontally
    brightness_range=[0.8, 1.2], # Adjust brightness between 80% and 120%
    fill_mode='nearest'        # Fill pixels with nearest value when shifting
)

In [ ]:
# Example of applying augmentation during training
train_generator = datagen.flow(X_train, y_train_cat, batch_size=32)

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate= 0.001),
                loss='binary_crossentropy',
                #loss="sparse_categorical_crossentropy",
                metrics=['accuracy', 'categorical_accuracy'])



history = model.fit(
    x=X_train,
    y=y_train_cat,
    batch_size=32,  # Reduced batch size for better generalization
    epochs=30,      # Increased epochs since we have early stopping
    validation_split=0.2,
    class_weight=class_weights_dict,  # Pass the dictionary here
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(X_test)
# Convert predictions to binary class labels 
y_pred  = y_pred_prob.argmax(axis=1)

In [ ]:
from sklearn.metrics  import classification_report
print('Classification Report:')
print(classification_report(y_test,y_pred))

In [ ]:
epochs = history.epoch

In [ ]:
accuracy_values = history.history['accuracy']
val_accuracy_values = history.history['val_accuracy']

plt.figure(figsize=(8, 4))
plt.plot(epochs, accuracy_values, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy_values, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm  = confusion_matrix(y_test, y_pred)
cmdisp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(5, 5))
cmdisp.plot(include_values=True, cmap="viridis", ax=ax, xticks_rotation="vertical")
plt.show()

In [ ]:
import numpy as np
import cv2

# Define class labels
class_labels = ["Normal", "Cataract", "Glaucoma"]

def preprocess_image(image_path):
    """
    Load and preprocess an image for model prediction.
    """
    img = cv2.imread(image_path)  # Load image
    img = cv2.resize(img, (100, 100))  # Resize to match model input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert color format
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image(image_path, model):
    """
    Predict the class of a given retina image.
    """
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    return class_labels[predicted_class], confidence

# Example usage:
test_image_path = "/Users/edwardtyler/Desktop/eye-images_2/2_cataract/cataract_047.png"  # Change to actual test image path
predicted_label, confidence = predict_image(test_image_path, model)

print(f"Predicted Class: {predicted_label}")
print(f"Confidence Score: {confidence:.2f}")